In [ ]:
!pip install pyspark

     |████████████████████████████████| 212.3MB 71kB/s 
     |████████████████████████████████| 204kB 19.7MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=39b2e3e39352a6ea8181159a779bb7b4e799d085a03a33c7b37ac1a18fb7dbb6
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from pyspark.sql import SparkSession as ss

spark = ss.builder.getOrCreate()

In [ ]:
from pyspark.sql.types import StringType, DoubleType, StructType, StructField
from pyspark.ml.linalg import VectorUDT
# u_id: string (nullable = true)
#  |-- USE_embed: vector (nullable = true)
#  |-- mean_sent_score: double (nullable = true)
#  |-- final_sent_class: string (nullable = true)
#  |-- Train/Test: string (nullable = false)



# schema = StructType([StructField('USE_embed', VectorUDT(), True),
#                      StructField('mean_sent_score', DoubleType(), True),
#                      StructField('final_sent_class', StringType(), True),
#                      StructField('Train/Test',StringType(),True)])

In [ ]:
df = spark.read.parquet('/content/drive/My Drive/Sentence Embeddings/USE_EMBED_mod_all_final',header = True,multiLine = True,inferSchema = True)

In [ ]:
df1 = df.select("USE_embed","Train/Test","final_sent_class")

In [ ]:
df1.show()

+--------------------+----------+----------------+
|           USE_embed|Train/Test|final_sent_class|
+--------------------+----------+----------------+
|[-0.0447620600461...|Train_Data|             Pos|
|[-0.0031811974477...|Train_Data|             Pos|
|[0.02649729140102...|Train_Data|             Neg|
|[0.00219017313793...|Train_Data|             Neg|
|[0.03250433504581...|Train_Data|             Neg|
|[-0.0042915488593...|Train_Data|             Neu|
|[0.03137646242976...|Train_Data|            VPos|
|[0.01750919222831...|Train_Data|             Neg|
|[-0.0673646926879...|Train_Data|             Neu|
|[-0.0862222686409...|Train_Data|             Pos|
|[-0.0054600276052...|Train_Data|             Neu|
|[0.05736451968550...|Train_Data|            VNeg|
|[-0.0177575312554...|Train_Data|             Pos|
|[0.05297778546810...|Train_Data|             Neg|
|[0.01868452318012...|Train_Data|             Neu|
|[0.01999289356172...|Train_Data|             Neu|
|[0.03525874018669...|Train_Dat

In [ ]:
from pyspark.ml.feature import PCA

pca = PCA(k=200, inputCol="USE_embed", outputCol="pcaFeatures")
model = pca.fit(df1)

result = model.transform(df1).select("pcaFeatures","Train/Test","final_sent_class")
result.show(truncate=False)


In [ ]:
from pyspark.ml.feature import StringIndexer
class_indexer = StringIndexer(inputCol="final_sent_class", outputCol="target")
set_indexer = StringIndexer(inputCol="Train/Test", outputCol="Train/Test_numeric")
res = class_indexer.fit(result).transform(result)
res_f = set_indexer.fit(res).transform(res)
res_f.show()

+--------------------+----------+----------------+------+------------------+
|         pcaFeatures|Train/Test|final_sent_class|target|Train/Test_numeric|
+--------------------+----------+----------------+------+------------------+
|[-0.0809263841557...|Train_Data|             Pos|   0.0|               0.0|
|[0.25014448951087...|Train_Data|             Pos|   0.0|               0.0|
|[-0.3692611182932...|Train_Data|             Neg|   1.0|               0.0|
|[0.12337869379586...|Train_Data|             Neg|   1.0|               0.0|
|[-0.0837672505061...|Train_Data|             Neg|   1.0|               0.0|
|[-0.1148557734415...|Train_Data|             Neu|   2.0|               0.0|
|[0.01599825095628...|Train_Data|            VPos|   3.0|               0.0|
|[-0.0673015852529...|Train_Data|             Neg|   1.0|               0.0|
|[-0.1048613737303...|Train_Data|             Neu|   2.0|               0.0|
|[-0.4277537223995...|Train_Data|             Pos|   0.0|               0.0|

In [ ]:
from pyspark.ml.feature import VectorAssembler
inputCols = [
 "pcaFeatures",
 "target",
 "Train/Test_numeric"
]
outputCol = "features"
df_va = VectorAssembler(inputCols = inputCols, outputCol = outputCol)
df_new = df_va.transform(res_f)


In [ ]:
x = [-0.08092638415574055,0.15166664745191005,-0.17097584719393696,-0.10766566593892939,0.026162333740846214,0.21000998469263454,0.08406478458145886,-0.016787843365595363,-0.05281778973223864,-0.13561125657326442,-0.15784140174257139,-0.16491373638262397,-0.03649214928145408,0.20487370476842387,0.25484367282906517,-0.053793184809303035,0.12754842059188778,0.06362132507461153,0.0839338250384004,0.2418652592834569,0.0,0.0]
len(x)
# df_new.select("features").show(truncate = False)

22

In [ ]:
from pyspark.ml.functions import vector_to_array
from pyspark.sql.functions import col

df2_p1 = df_new.withColumn("xs", vector_to_array("features")).select([col("xs")[i] for i in range(0,20)])
df2_p2 = df_new.withColumn("xs", vector_to_array("features")).select([col("xs")[i] for i in range(20,40)])
df2_p3 = df_new.withColumn("xs", vector_to_array("features")).select([col("xs")[i] for i in range(40,52)])
# df2_p4 = df_new.withColumn("xs",vector_to_array("features")).select([col("xs")[i] for i in range(75,101)])

In [ ]:
df2_p3.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------+------+
|              xs[40]|              xs[41]|              xs[42]|              xs[43]|              xs[44]|              xs[45]|              xs[46]|              xs[47]|              xs[48]|              xs[49]|xs[50]|xs[51]|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------+------+
|-0.01440190177587...| 0.07327324695618691|-0.00857496204046...| 0.11785529309814458|-0.04738044870307...|0.004800771490565853| 0.07768097438331885|-0.05359218659177853|0.025458331815397712| 0.18578204504399076|   0.0|   0.0|
| 0.10708381843017085|0.003108615194534934|-0.02438550753070579|-0.01679864423350...|0.001979718

In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *

w=Window.orderBy(lit(1))

df4=df2_p1.withColumn("rn",row_number().over(w)-1)
df3=res.select("u_id").withColumn("rn",row_number().over(w)-1)

df4 = df4.join(df3,["rn"]).drop("rn")

In [ ]:
df2_p1.printSchema()

root
 |-- xs[0]: double (nullable = true)
 |-- xs[1]: double (nullable = true)
 |-- xs[2]: double (nullable = true)
 |-- xs[3]: double (nullable = true)
 |-- xs[4]: double (nullable = true)
 |-- xs[5]: double (nullable = true)
 |-- xs[6]: double (nullable = true)
 |-- xs[7]: double (nullable = true)
 |-- xs[8]: double (nullable = true)
 |-- xs[9]: double (nullable = true)
 |-- xs[10]: double (nullable = true)
 |-- xs[11]: double (nullable = true)
 |-- xs[12]: double (nullable = true)
 |-- xs[13]: double (nullable = true)
 |-- xs[14]: double (nullable = true)
 |-- xs[15]: double (nullable = true)
 |-- xs[16]: double (nullable = true)
 |-- xs[17]: double (nullable = true)
 |-- xs[18]: double (nullable = true)
 |-- xs[19]: double (nullable = true)
 |-- xs[20]: double (nullable = true)
 |-- xs[21]: double (nullable = true)



In [ ]:
ls = [-0.11553205251181606,-0.2593063849344816,0.027018887690345657,0.04465626242613705,-0.04112241663777816,0.30816824322783715,-0.03322550047120177,-0.028026552667863663,0.06377391681507573,0.18872072350627198,0.28358594089314215,-0.030165726853164283,0.05466480366314219,0.13862864878824646,-0.02524396261542477,0.029972463495577157,0.025260833339073738,0.05123203109072145,-0.06768830240470275,-0.014843957126907292,-0.1518633817970781,-0.14506551263602793,0.04966831692644326,-0.08558779734547567,-0.22358904506305066,-0.1795291501647179,0.1834087777932504,-5.938075151134263E-4,-0.11224060962982435,0.1351318018269279,0.04108651459511949,-0.14776377189247633,-0.0055304397122846105,-0.00765802385691248,0.0982566855719619,-0.20320660747370964,-0.17247527923010275,-0.09491252370431198,-0.007219139957646644,0.08283062170689293]

In [ ]:
import numpy as np
num_p1 = np.array(df2_p1.select('*').collect())
num_p2 = np.array(df2_p2.select("*").collect())
num_p3 = np.array(df2_p3.select("*").collect())
# num_p4 = np.array(df2_p4.select("*").collect())

In [ ]:
num_p3.shape

(1700330, 12)

In [ ]:
rs = np.concatenate((num_p1,num_p2,num_p3),axis = 1)

In [ ]:
rs.shape

(1700330, 52)

In [ ]:
x = np.array(ls)

In [ ]:
x.reshape((8,5))

array([[-0.11553205, -0.25930638,  0.02701889,  0.04465626, -0.04112242],
       [ 0.30816824, -0.0332255 , -0.02802655,  0.06377392,  0.18872072],
       [ 0.28358594, -0.03016573,  0.0546648 ,  0.13862865, -0.02524396],
       [ 0.02997246,  0.02526083,  0.05123203, -0.0676883 , -0.01484396],
       [-0.15186338, -0.14506551,  0.04966832, -0.0855878 , -0.22358905],
       [-0.17952915,  0.18340878, -0.00059381, -0.11224061,  0.1351318 ],
       [ 0.04108651, -0.14776377, -0.00553044, -0.00765802,  0.09825669],
       [-0.20320661, -0.17247528, -0.09491252, -0.00721914,  0.08283062]])

In [ ]:
import pickle

pickle_out = open("/content/drive/My Drive/Sentence Embeddings/numpy_50D_with_labels.pickle","wb")
pickle.dump(rs, pickle_out)
pickle_out.close()

In [ ]:
import sklearn.datasets
import pandas as pd
import numpy as np
import umap


In [ ]:
# pendigits = sklearn.datasets.load_digits()
# mnist = sklearn.datasets.fetch_openml('mnist_784')
fmnist = sklearn.datasets.fetch_openml('Fashion-MNIST')

In [ ]:
type(fmnist.data)

numpy.ndarray

In [ ]:
from sklearn.datasets import fetch_openml
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Dimension reduction and clustering libraries
import umap
# import hdbscan
import sklearn.cluster as cluster
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score


In [ ]:
mnist = fetch_openml('Fashion-MNIST', version=1)
mnist.target = mnist.target.astype(int)


In [ ]:
mnist

{'DESCR': "**Author**: Han Xiao, Kashif Rasul, Roland Vollgraf  \n**Source**: [Zalando Research](https://github.com/zalandoresearch/fashion-mnist)  \n**Please cite**: Han Xiao and Kashif Rasul and Roland Vollgraf, Fashion-MNIST: a Novel Image Dataset for Benchmarking Machine Learning Algorithms, arXiv, cs.LG/1708.07747  \n\nFashion-MNIST is a dataset of Zalando's article images, consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes. Fashion-MNIST is intended to serve as a direct drop-in replacement for the original MNIST dataset for benchmarking machine learning algorithms. It shares the same image size and structure of training and testing splits. \n\nRaw data available at: https://github.com/zalandoresearch/fashion-mnist\n\n### Target classes\nEach training and test example is assigned to one of the following labels:\nLabel  Description  \n0  T-shirt/top  \n1  Trouser  \n2  P